In [18]:
import os
import json
import time
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field
import requests

# jsonschema imports
from jsonschema import validate, ValidationError, FormatChecker

import openai
import requests
from openai import OpenAI

In [19]:
def create_openai_client_for_groq():
    """
    Create an OpenAI-compatible client for Groq API.
    Requires GROQ_API_KEY in your environment.
    """
    if OpenAI is None:
        raise RuntimeError("OpenAI client library not installed or OpenAI class not available.")
    base_url = "https://api.groq.com/openai/v1"
    api_key = os.environ.get("GROQ_API_KEY")
    if not api_key:
        raise RuntimeError("Set GROQ_API_KEY as an environment variable before using this cell.")
    client = OpenAI(api_key=api_key, base_url=base_url)
    return client

# -------------------------
# Raw requests wrapper
# -------------------------
def groq_chat_request_raw(
    messages: List[Dict[str, str]],
    model: str = "llama-3.3-70b-versatile",
    max_tokens: int = 512,
    temperature: float = 0.0,
):
    """
    Send raw POST request to Groq OpenAI-compatible endpoint.
    Returns parsed JSON on success or raises HTTPError.
    """
    api_key = os.environ.get("GROQ_API_KEY")
    if not api_key:
        raise RuntimeError("Set GROQ_API_KEY in env before running.")
    endpoint = "https://api.groq.com/openai/v1/chat/completions"
    payload = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
    }
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    # debug print (uncomment if needed):
    # print("DEBUG: POST", endpoint, "payload model:", model)
    resp = requests.post(endpoint, headers=headers, json=payload)
    resp.raise_for_status()
    return resp.json()


In [20]:
@dataclass
class ConversationHistory:
    raw_messages: List[Dict[str, str]] = field(default_factory=list)
    summary: Optional[str] = None
    runs_since_last_summary: int = 0

    def append(self, role: str, content: str):
        self.raw_messages.append({"role": role, "content": content})
        self.runs_since_last_summary += 1

    def truncate_by_turns(self, n: int) -> List[Dict[str,str]]:
        return self.raw_messages[-n:]

    def truncate_by_chars(self, max_chars: int) -> List[Dict[str,str]]:
        out = []
        total = 0
        for msg in reversed(self.raw_messages):
            l = len(msg['content'])
            if total + l > max_chars and out:
                break
            out.append(msg)
            total += l
        return list(reversed(out))

    def truncate_by_words(self, max_words: int) -> List[Dict[str,str]]:
        out = []
        total = 0
        for msg in reversed(self.raw_messages):
            w = len(msg['content'].split())
            if total + w > max_words and out:
                break
            out.append(msg)
            total += w
        return list(reversed(out))

    def maybe_summarize(self, k: int, summarizer_fn):
        """
        If runs_since_last_summary >= k, call summarizer_fn(conversation) -> summary string,
        store summary and replace earlier history (we'll keep the summary as a system message).
        """
        if self.runs_since_last_summary >= k:
            combined = "\n".join([f"{m['role']}: {m['content']}" for m in self.raw_messages])
            new_summary = summarizer_fn(combined)
            tail = self.truncate_by_turns(3)
            self.raw_messages = [{"role": "system", "content": f"SUMMARY: {new_summary}"}] + tail
            self.summary = new_summary
            self.runs_since_last_summary = 0
            return True
        return False

In [21]:
def naive_summarizer(text: str, max_sentences: int = 3) -> str:
    import re
    sents = re.split(r'(?<=[.!?])\s+', text.strip())
    return " ".join(sents[:max_sentences]) if sents else ""

def groq_summarizer(text: str, model: str = "llama-3.3-70b-versatile"):
    """
    Use Groq to summarize. If no API key / client, returns a mock string.
    """
    try:
        # Try raw request since it's generic
        messages = [
            {"role": "system", "content": "You are a concise summarizer. Produce a short paragraph summary (max 60 words)."},
            {"role": "user", "content": f"Summarize the following conversation:\n\n{text}"}
        ]
        resp = groq_chat_request_raw(messages, model=model, max_tokens=150)
        return resp['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"[MOCK SUMMARY: {str(e)}]"


In [22]:
info_schema = {
    "type": "object",
    "properties": {
        "name": {"type": ["string", "null"]},
        "email": {"type": ["string", "null"], "format": "email"},
        "phone": {"type": ["string", "null"]},
        "location": {"type": ["string", "null"]},
        "age": {"type": ["integer", "null"], "minimum": 0, "maximum": 120}
    },
    # we do not require every field to be present; allow nulls
    "required": ["name"],
    "additionalProperties": False
}


In [23]:
def call_groq_for_extraction(user_chat: str, model: str = "llama-3.3-70b-versatile"):
    """
    Ask Groq to return ONLY a JSON object (name, email, phone, location, age).
    Returns the parsed object or raises on failure.
    If GROQ_API_KEY is not set, returns a mocked parsed dict for offline testing.
    """
    api_key = os.environ.get("GROQ_API_KEY")
    if not api_key:
        # mocked output for offline demonstration
        # try to do a heuristic parse for better demo
        import re
        name = None
        if re.search(r"abhijit", user_chat, re.I):
            name = "Abhijit Rajkumar"
        elif re.search(r"aisha", user_chat, re.I):
            name = "Aisha"
        email_m = re.search(r"[\w\.-]+@[\w\.-]+\.\w+", user_chat)
        phone_m = re.search(r"(?:\+?\d{1,3}[-\s]?)?\d{6,12}", user_chat)
        age_m = re.search(r"\b(\d{1,3})\b", user_chat)
        return {
            "name": name or None,
            "email": email_m.group(0) if email_m else None,
            "phone": phone_m.group(0) if phone_m else None,
            "location": None,
            "age": int(age_m.group(1)) if age_m and 0 <= int(age_m.group(1)) <= 120 else None
        }

    # If API key is present, call Groq
    messages = [
        {"role": "system", "content": "You must output a JSON object and nothing else. Use null for missing fields."},
        {"role": "user", "content": f"Chat: ```{user_chat}```\n\nExtract name, email, phone, location, age. Respond only with JSON."}
    ]
    resp = groq_chat_request_raw(messages, model=model, max_tokens=200)
    # parse
    try:
        content = resp['choices'][0]['message']['content']
    except Exception:
        raise RuntimeError("Unexpected response structure from Groq: " + json.dumps(resp)[:1000])

    # Some models sometimes wrap JSON in backticks or markdown; sanitize:
    text = content.strip()
    # find first '{' and last '}' and extract
    first = text.find("{")
    last = text.rfind("}")
    if first != -1 and last != -1 and last > first:
        text = text[first:last+1]
    try:
        parsed = json.loads(text)
    except json.JSONDecodeError as e:
        raise RuntimeError("Failed to parse JSON from model output: " + str(e) + "\nOutput:\n" + content)

    return parsed

In [24]:
if __name__ == "__main__":
    # Create history and sample messages
    history = ConversationHistory()
    sample_msgs = [
        ("user", "Hi, I'm Abhijit. I'm looking for AI/ML backend roles."),
        ("assistant", "Great — what's your primary tech stack?"),
        ("user", "Python, FastAPI, HuggingFace, Qdrant, Docker."),
        ("assistant", "Do you have experience with AWS and Kafka?"),
        ("user", "Yes, some with AWS and Kafka; built a small pipeline."),
        ("assistant", "What's your preferred location?"),
        ("user", "India or fully remote."),
        ("assistant", "Thanks — I'll search and prepare tailored applications.")
    ]
    for role, text in sample_msgs:
        history.append(role, text)

    print("Full history (raw):")
    print(json.dumps(history.raw_messages, indent=2))

    print("\nTruncate by last 4 turns:")
    print(json.dumps(history.truncate_by_turns(4), indent=2))

    print("\nTruncate by chars (max 120 chars):")
    print(json.dumps(history.truncate_by_chars(120), indent=2))

    print("\nTruncate by words (max 40 words):")
    print(json.dumps(history.truncate_by_words(40), indent=2))

    # Demonstrate k-th summarization after every 3 runs using naive summarizer
    print("\nDemonstrate periodic summarization (k=3) using naive summarizer.")
    for i in range(3):
        history.append("user", f"Extra message {i+1}")
    triggered = history.maybe_summarize(3, lambda t: naive_summarizer(t, max_sentences=2))
    print("Summarization triggered:", triggered)
    print("History after summarization:")
    print(json.dumps(history.raw_messages, indent=2))
    print("Stored summary:", history.summary)

    # Task 2 demos: sample chats + extraction + validation
    sample_chats = [
        "Hi, this is Abhijit Rajkumar. My email is abhijit.raj@example.com and my phone is +91-9876543210. I'm 30 and based in Bengaluru, India.",
        "Hello, I'm Aisha. Reach me at aisha@example.co. I'm currently in Mumbai.",
        "This is a short message — no contact info here."
    ]

    for chat in sample_chats:
        print("\nChat:", chat)
        try:
            extracted = call_groq_for_extraction(chat)
            print("Extracted:", extracted)
            # Validate
            try:
                validate(instance=extracted, schema=info_schema, format_checker=FormatChecker())
                print("Validation: OK ✅")
            except ValidationError as ve:
                print("Validation: FAILED ❌ -", ve.message)
        except Exception as e:
            print("Error while extracting:", str(e))

Full history (raw):
[
  {
    "role": "user",
    "content": "Hi, I'm Abhijit. I'm looking for AI/ML backend roles."
  },
  {
    "role": "assistant",
    "content": "Great \u2014 what's your primary tech stack?"
  },
  {
    "role": "user",
    "content": "Python, FastAPI, HuggingFace, Qdrant, Docker."
  },
  {
    "role": "assistant",
    "content": "Do you have experience with AWS and Kafka?"
  },
  {
    "role": "user",
    "content": "Yes, some with AWS and Kafka; built a small pipeline."
  },
  {
    "role": "assistant",
    "content": "What's your preferred location?"
  },
  {
    "role": "user",
    "content": "India or fully remote."
  },
  {
    "role": "assistant",
    "content": "Thanks \u2014 I'll search and prepare tailored applications."
  }
]

Truncate by last 4 turns:
[
  {
    "role": "user",
    "content": "Yes, some with AWS and Kafka; built a small pipeline."
  },
  {
    "role": "assistant",
    "content": "What's your preferred location?"
  },
  {
    "role": "us